In [71]:
from functools import partial
from typing import List, Tuple

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import HTML, display, update_display

In [72]:
class Colors:
    START = 'green'
    GOAL = 'blue'
    OBSTACLE = 'black'
    PATH = 'red'


class Interface:
    def __init__(self, shape=(10,10)):
        self.n_rows, self.n_cols = shape
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='50px', height='50px')
                )
                for _ in range(self.n_cols)
            ]
            for _ in range(self.n_rows)
        ]

        self.map_widget = VBox([HBox(row) for row in self.buttons])
        self.link_positions()

        self.start_pos, self.goal_pos = None, None
        self.obstacles = set()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the map

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if self.start_pos is None:
            self.start_pos = pos
            button.style.button_color = Colors.START

        elif self.goal_pos is None:
            self.goal_pos = pos
            button.style.button_color = Colors.GOAL

        else:
            self.obstacles.add(pos)
            button.style.button_color = Colors.OBSTACLE

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True

    def start(self):
        """Display map interface"""
        display(self.map_widget)



In [73]:
interface = Interface()
interface.start()

In [74]:
MOVES = [(+1, 0), (-1, 0), (0, +1), (0, -1)]


def BFS(start_pos: Tuple[int, int], goal_pos: Tuple[int, int],
        obstacles: List[Tuple[int, int]], max_y: int, max_x: int):
    """Breadth-first search algorithm"""

    def is_valid(position: Tuple[int, int]):
        """Checks if a position is valid
        i.e. it is inside map and it is not a obstacle"""
        y, x = position
        return (x >= 0 and x < max_x and
                y >= 0 and y < max_y and
                position not in obstacles)

    def get_next_positions(y, x):
        """Get next possible moves"""
        return [
            new_pos for dy, dx in MOVES
            if is_valid(new_pos := (y + dy, x + dx))
        ]

    visited_positions = set()
    to_visit = [(start_pos, [])]  # position, path_to_position

    while to_visit:
        pos, path = to_visit.pop(0)

        if pos == goal_pos:
            return path

        if pos in visited_positions:
            continue

        visited_positions.add(pos)
        to_visit += [(next_pos, [*path, pos])
                     for next_pos in get_next_positions(*pos)]

In [80]:
class PathFinder(Interface):
    def __init__(self, shape=(10, 10)):
        super().__init__(shape)
        self.find_button = widgets.Button(
            description='Find shortest path',
            style=widgets.ButtonStyle(button_color=Colors.START)
        )

        self.find_button.on_click(self.search)

    def set_path(self, path: List[Tuple[int, int]]):
        """Color a path in map"""
        for y, x in path[1:]:
            self.buttons[y][x].style.button_color = Colors.PATH

    def search(self, *args):
        """Disable editions on map, then finds the shorterst path and draw it"""
        self.disable_buttons()
        path = BFS(self.start_pos, self.goal_pos, self.obstacles, self.n_rows,
                   self.n_cols)
        self.set_path(path)
        return path

    def start(self):
        """Display map interface"""
        display(VBox([self.map_widget, self.find_button]))



path_finder = PathFinder()
path_finder.start()